# Appendix E in Master Thesis Aad Vijn 
# Magnetized Steel Plates - Plotting the Magnetic Field Components  

Here we explore Appendix E of [Vijn's master thesis](https://repository.tudelft.nl/islandora/object/uuid%3A89f31ad6-e8c4-4db5-ac6e-eb6fdeb3f7e0?collection=education). 

## Import Packages 

In [1]:
using StaticArrays
using LinearAlgebra

using BenchmarkTools 

using Plots

## Section 1: Plate and Air Surrounding Plate

In [56]:
const Point3D = SVector{3,Float64}

struct Plate 
    L::Float64 
    W::Float64 
    T::Float64 
end

#..set dimension of the plate 
L = 1. 
W = 1
T = 0.05 

#..initialize the plate 
plate = Plate(L,W,T)

#..set dimension of the box of air around the plate 
Lair = 2*L
Wair = 2*W

#..construct uniform mesh on the air domain 
LVec = Vector(-Lair:Lair/50:Lair)
WVec = Vector(-Wair:Wair/50:Wair) 
R = [ Point3D(x,y,0) for x in LVec, y in WVec ];

## Section 2: Analytical Expressions for the Magnetic Flux Components 

In [34]:
function F1(r,t,plate)
    W = plate.W
    L = plate.L
    denom1 = (r.x+L/2)^2+(r.z)^2
    numer2 = r.y - W/2*t
    denom2 = sqrt( (r.x+L/2)^2+(r.y+W/2*t)^2+(r.z)^2 )
    return 1/denom1*numer2/denom2 
end 

function F2(r,t,plate)
    W = plate.W
    L = plate.L
    denom1 = (r.x-L/2)^2+(r.z)^2
    numer2 = r.y - W/2*t
    denom2 = sqrt( (r.x-L/2)^2+(r.y+W/2*t)^2+(r.z)^2 )
    return 1/denom1*numer2/denom2 
end 

function F3(r,t,plate)
    W = plate.W
    L = plate.L
    denom1 = sqrt( (r.x+L/2)^2+(r.y-W/2*t)^2+(r.z)^2 )
    return 1/denom1 
end 

function F4(r,t,plate)
    W = plate.W
    L = plate.L
    denom1 = sqrt( (r.x-L/2)^2+(r.y-W/2*t)^2+(r.z)^2 )
    return 1/denom1 
end 

function I1(r,t,plate)
    W = plate.W
    return 2/W*( F1(r,1.,plate) - F1(r,-1.,plate) )
end 

function I2(r,t,plate)
    W = plate.W
    return 2/W*( F2(r,1.,plate) - F2(r,-1.,plate) )
end 

function I3(r,t,plate)
    W = plate.W
    return 2/W*( F3(r,1.,plate) - F3(r,-1.,plate) )
end 

function I4(r,t,plate)
    W = plate.W
    return 2/W*( F4(r,1.,plate) - F4(r,-1.,plate) )
end 

function Bx(r,plate)
    mu0 = 4*pi*1e-7
    I1 = F1(r,1.,plate) - F1(r,-1.,plate)
    I2 = F2(r,1.,plate) - F2(r,-1.,plate)
    val = (r.x+L/2)*I1 - (r.x-L/2)*I2 
    return val 
end 

function By(r,plate)
    mu0 = 4*pi*1e-7
    I3 = F3(r,1.,plate) - F3(r,-1.,plate)
    I4 = F4(r,1.,plate) - F4(r,-1.,plate)
    val = I3 - I4 
    return val 
end

By (generic function with 1 method)

## Section 3: Evaluation and Plotting of the Analytical Expressions

In [ ]:
x = [ r.x for r in R]
y = [ r.y for r in R]
valx = [ Bx(r, plate) for r in R]
valy = [ By(r, plate) for r in R]
p1 = contour(valx)
p2 = contour(valy)
plot(p1,p2)